In [2]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
from datetime import datetime
import calendar
import fiona

# Read GEE data

In [50]:
dir_gee = r"C:\Users\tangz\Desktop\US_CornSoy_ClassificationYield\GEEData"
PATH_S2 = glob.glob(os.path.join(dir_gee, "S2*.csv"))
PATH_S2

['C:\\Users\\tangz\\Desktop\\US_CornSoy_ClassificationYield\\GEEData\\s2_ts_2021_IL.csv',
 'C:\\Users\\tangz\\Desktop\\US_CornSoy_ClassificationYield\\GEEData\\s2_ts_2022_IL.csv',
 'C:\\Users\\tangz\\Desktop\\US_CornSoy_ClassificationYield\\GEEData\\s2_ts_2023_IL.csv']

In [51]:
df_2023 = pd.read_csv('C:\\Users\\tangz\\Desktop\\US_CornSoy_ClassificationYield\\GEEData\\s2_ts_2023_IL.csv')

In [37]:
path_csv = r"C:\Users\tangz\Downloads\s2_ts_2022.csv"
df = pd.read_csv(path_csv)
# clean 
# system:index column convert to date using the first 10 characters of the index
df['Date'] = df['system:index'].str[:8]
# convert to datetime
df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d')
# Drop system:index column
df = df.drop(columns=['system:index'])
# Drop .geo column
df = df.drop(columns=['.geo'])
#change colunm type of CSBACRES and CSBID
df['CSBACRES'] = df['CSBACRES'].astype('float64')
df['CSBID'] = df['CSBID'].astype('str')
df.head()

,B11,B12,B2,B3,B4,B5,B6,B7,B8,CNTY,CSBACRES,CSBID,Date
0,0.084938,0.087876,0.938261,0.910396,0.910328,0.914724,0.890018,0.859706,0.909358,McLean,16.104088,171522007807993,2022-01-03
1,0.088674,0.090536,0.860714,0.846939,0.853107,0.861001,0.841714,0.818077,0.867266,McLean,31.945160,171522007807810,2022-01-03
2,0.102222,0.104262,0.793342,0.792903,0.811309,0.819792,0.803143,0.784491,0.832835,McLean,5.224200,171522007807829,2022-01-03
3,0.089174,0.090322,0.829021,0.816888,0.826646,0.836504,0.818319,0.796927,0.844610,McLean,46.208765,171522007807867,2022-01-03
4,0.083845,0.082804,0.672182,0.657361,0.661786,0.681860,0.662598,0.648355,0.684199,McLean,4.031501,171522007808186,2022-01-03


# Read Geodata

In [6]:
# dir_csb = r"C:\Users\tangz\Desktop\US_CornSoy_ClassificationYield\GeoData\CSB1522.gdb"
# layers = fiona.listlayers(dir_csb)
# for layer in layers:
#     print(layer)

nationalGIS


In [7]:
IL = gpd.read_file(r"C:\Users\tangz\Desktop\US_CornSoy_ClassificationYield\GeoData\Vectors\csb_IL_2022.shp")

In [38]:
Mclean = gpd.read_file(r"C:\Users\tangz\Desktop\US_CornSoy_ClassificationYield\GeoData\Vectors\csb_IL_2022_McLean.shp")
Mclean['CSBID'] = Mclean['CSBID'].astype('str')

In [46]:
print(IL.shape);print(Mclean.shape)

(653148, 22)
(9720, 22)


In [47]:
653148/9720

67.1962962962963

# Merge together

In [45]:
# Merge with the GEE data
select_columns = ['CSBID', 'CSBACRES', 'R15', 'R16', 'R17', 'R18', 'R19','R20','R21','R22','STATEASD', 'CNTY']
df_Mclean = pd.merge(df, Mclean[select_columns], how='left', on=['CNTY', 'CSBACRES', 'CSBID'])
print("How many NAs in the dataframe: " + str(df_Mclean.isnull().sum().sum()))
df_Mclean.head()

How many NAs in the dataframe: 0


,B11,B12,B2,B3,B4,B5,B6,B7,B8,CNTY,...,Date,R15,R16,R17,R18,R19,R20,R21,R22,STATEASD
0,0.084938,0.087876,0.938261,0.910396,0.910328,0.914724,0.890018,0.859706,0.909358,McLean,...,2022-01-03,1,5,24,1,5,5,1,229,1740
1,0.088674,0.090536,0.860714,0.846939,0.853107,0.861001,0.841714,0.818077,0.867266,McLean,...,2022-01-03,1,5,1,5,1,5,1,5,1740
2,0.102222,0.104262,0.793342,0.792903,0.811309,0.819792,0.803143,0.784491,0.832835,McLean,...,2022-01-03,1,5,5,5,5,5,1,5,1740
3,0.089174,0.090322,0.829021,0.816888,0.826646,0.836504,0.818319,0.796927,0.844610,McLean,...,2022-01-03,1,5,1,5,1,5,1,5,1740
4,0.083845,0.082804,0.672182,0.657361,0.661786,0.681860,0.662598,0.648355,0.684199,McLean,...,2022-01-03,1,5,1,5,5,5,1,5,1740


In [10]:
Mclean.columns

Index(['CSBID', 'CSBYEARS', 'CSBACRES', 'R15', 'R16', 'R17', 'R18', 'R19',
       'R20', 'R21', 'R22', 'STATEFIPS', 'STATEASD', 'ASD', 'CNTY', 'CNTYFIPS',
       'INSIDE_X', 'INSIDE_Y', 'Shape_Leng', 'Shape_Le_1', 'Shape_Area',
       'geometry'],
      dtype='object')

In [8]:
print(IL.shape)
print(IL['CSBID'].unique().shape)

(653148, 22)
(653148,)


In [5]:
Mclean.columns

Index(['CSBID', 'CSBYEARS', 'CSBACRES', 'R15', 'R16', 'R17', 'R18', 'R19',
       'R20', 'R21', 'R22', 'STATEFIPS', 'STATEASD', 'ASD', 'CNTY', 'CNTYFIPS',
       'INSIDE_X', 'INSIDE_Y', 'Shape_Leng', 'Shape_Le_1', 'Shape_Area',
       'geometry'],
      dtype='object')